כדי להריץ את המחברות הבאות, אם עדיין לא עשית זאת, עליך להגדיר את מפתח ה-openai בתוך קובץ .env כ-`OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

לאחר מכן, נטען את אינדקס ההטמעה לתוך Dataframe של Pandas. אינדקס ההטמעה מאוחסן בקובץ JSON בשם `embedding_index_3m.json`. אינדקס ההטמעה מכיל את ההטמעות עבור כל התמלילים של YouTube עד סוף אוקטובר 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

לאחר מכן, ניצור פונקציה בשם `get_videos` שתבצע חיפוש באינדקס ההטמעה עבור השאילתה. הפונקציה תחזיר את 5 הסרטונים העליונים שהכי דומים לשאילתה. הפונקציה פועלת כך:

1. תחילה, נוצרת עותק של אינדקס ההטמעה.
2. לאחר מכן, מחושבת ההטמעה עבור השאילתה באמצעות ממשק ה-API של OpenAI Embedding.
3. אז נוצר עמודה חדשה באינדקס ההטמעה בשם `similarity`. עמודת ה-`similarity` מכילה את הדמיון הקוסינוסי בין ההטמעה של השאילתה לבין ההטמעה של כל קטע וידאו.
4. לאחר מכן, מסונן אינדקס ההטמעה לפי עמודת ה-`similarity`. האינדקס מסונן כך שיכלול רק סרטונים שהדמיון הקוסינוסי שלהם גדול או שווה ל-0.75.
5. לבסוף, האינדקס ממויין לפי עמודת ה-`similarity` ומוחזרים 5 הסרטונים העליונים.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

הפונקציה הזו מאוד פשוטה, היא רק מדפיסה את תוצאות שאילתת החיפוש.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. ראשית, אינדקס ההטמעה נטען לתוך Dataframe של Pandas.  
2. לאחר מכן, המשתמש מתבקש להזין שאילתה.  
3. אז קוראים לפונקציה `get_videos` כדי לחפש באינדקס ההטמעה את השאילתה.  
4. לבסוף, קוראים לפונקציה `display_results` כדי להציג את התוצאות למשתמש.  
5. לאחר מכן המשתמש מתבקש להזין שאילתה נוספת. תהליך זה נמשך עד שהמשתמש מזין `exit`.  

![](../../../../translated_images/he/notebook-search.1e320b9c7fcbb0bc.png)  

תתבקש להזין שאילתה. הזן שאילתה ולחץ אנטר. האפליקציה תחזיר רשימת סרטונים שרלוונטיים לשאילתה. האפליקציה גם תחזיר קישור למקום בסרטון שבו נמצא התשובה לשאלה.  

הנה כמה שאילתות לנסות:  

- מה זה Azure Machine Learning?  
- איך רשתות עצביות קונבולוציוניות עובדות?  
- מהי רשת עצבית?  
- האם אפשר להשתמש ב-Jupyter Notebooks עם Azure Machine Learning?  
- מה זה ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון כי תרגומים אוטומטיים עלולים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפת המקור שלו נחשב למקור הסמכותי. למידע קריטי מומלץ להשתמש בתרגום מקצועי על ידי אדם. אנו לא נושאים באחריות לכל אי-הבנה או פרשנות שגויה הנובעת משימוש בתרגום זה.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
